# Settings and data loading
---

This cell is to be run prior to all others. It sets parameters for data loading, plotting and saving.
All other cells are structured to be authonomously run.

In [14]:
import os
import numpy as np
import matplotlib.pyplot as  plt 
from matplotlib import colormaps 
from scipy.optimize import curve_fit  
import importlib

#--------------------------------------------------------
def readTable1(dirIn="SPARCdata"):
    """
    Read data in "table1.dat" file in the dirIn folder, returns a list of dictionaries (named table).
    Each dict item is a row of the table, as in the description below. Dict keys correspond to labels in the table.

    ================================================================================
    Byte-by-byte Description of file: Table1.mrt
    --------------------------------------------------------------------------------
    Bytes Format Units         Label   Explanations
    --------------------------------------------------------------------------------
    1- 11 A11    ---           Galaxy  Galaxy Name
    12- 13 I2     ---           T       Hubble Type (1)
    14- 19 F6.2   Mpc           D       Distance
    20- 24 F5.2   Mpc         e_D       Mean error on D
    25- 26 I2     ---         f_D       Distance Method (2)
    27- 30 F4.1   deg           Inc     Inclination
    31- 34 F4.1   deg         e_Inc     Mean error on Inc
    35- 41 F7.3   10+9solLum    L[3.6]  Total Luminosity at [3.6]
    42- 48 F7.3   10+9solLum  e_L[3.6]  Mean error on L[3.6]
    49- 53 F5.2   kpc           Reff    Effective Radius at [3.6]
    54- 61 F8.2   solLum/pc2    SBeff   Effective Surface Brightness at [3.6]
    62- 66 F5.2   kpc           Rdisk   Disk Scale Length at [3.6]
    67- 74 F8.2   solLum/pc2    SBdisk  Disk Central Surface Brightness at [3.6]
    75- 81 F7.3   10+9solMass   MHI     Total HI mass
    82- 86 F5.2   kpc           RHI     HI radius at 1 Msun/pc2
    87- 91 F5.1   km/s          Vflat   Asymptotically Flat Rotation Velocity
    92- 96 F5.1   km/s        e_Vflat   Mean error on Vflat
    97- 99 I3     ---           Q       Quality Flag (3)
    100-113 A14    ---           Ref.    References for HI and Ha data (4)
    --------------------------------------------------------------------------------
    """
    filepath = os.path.join(dirIn, "Table1.dat")
    fileVar = open(filepath, 'r')
    counter = 0
    table = []
    for line in fileVar:
        if counter < 98 or line=="":
            pass
        else: 
            values = np.fromstring(line[11:-14], dtype=float, sep=' ')
            dictItem = {"name":     line[:11].lstrip(), 
                        "T":        int(  values[0]),
                        "D":        float(values[1]),
                        "e_D":      float(values[2]),
                        "f_D":      int(  values[3]),
                        "Inc":      float(values[4]),
                        "e_Inc":    float(values[5]),
                        "L[3.6]":   float(values[6]),
                        "e_L[3.6]": float(values[7]),
                        "Reff":     float(values[8]),
                        "SBeff":    float(values[9]),
                        "Rdisk":    float(values[10]),
                        "SBdisk":   float(values[11]),
                        "MHI":      float(values[12]),
                        "RHI":      float(values[13]),
                        "Vflat":    float(values[14]),
                        "e_Vflat":  float(values[15]),
                        "Q":        int(  values[16]),
                        "Ref":      line[-14:].lstrip()}
            table.append(dictItem)
        counter = counter +1
    return np.array(table)
#---------------------------------------------------------
def readRotmod(dirIn="SPARCdata/Rotmod_LTG"):
    """
    Inport data from "Rotmod_LTG" files. Files structure:
    Cols: Rad(kpc)	Vobs(km/s)	errV(km/s)	Vgas(km/s)	Vdisk(km/s)	Vbul(km/s)	SBdisk(L/pc^2)	SBbul(L/pc^2)	
    Output: list of data, each data is numpy array with cols as in files.
    """
    #input settings
    file_list= os.listdir(dirIn)

    #loading experimental data
    data_list  = [] #each entry is one file of data
    names_list = [] #save names of datasets, in same order as in data_list
    for file in file_list:
        filepath = os.path.join(dirIn, file)
        dataset = np.loadtxt(filepath, dtype=float)
        data_list.append(dataset)
        names_list.append(file[:-11])
    
    #return data 
    return data_list, names_list

#--- Data Loading -------------------------------------------
#load Table 1
table = readTable1()
#load data from Rotmod
data_list, names_list = readRotmod()


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_44796\2293382711.py:48: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  values = np.fromstring(line[11:-14], dtype=float, sep=' ')


# Visualize SPARC data

---

## 1. Velocity curves
The code plots the desired visualizations for each available galaxy and saves the results in a dedicated folder.<br>
This cell constructs velocity curves plots (Vobs, errV, Vgas, Vdisk, Vbul) as reported in the `Rotmod_LTG` files.

In [15]:
#output settings
dirOutPath = "SPARCplots/"
dirOut = "Experimental_RotationCurves"

#create output folder, if needed
os.makedirs(dirOutPath+dirOut, exist_ok=True)
#loop over dataset	
for i in range(len(data_list)):
	data = data_list[i]
	figname = names_list[i]+"_rotCurve"
	#figure
	plt.figure()
	plt.errorbar(data[:,0],data[:,1],data[:,2],marker='o',linestyle="dotted",label=r"$V_{Obs}$")
	plt.plot(data[:,0],data[:,3],marker='o',linestyle="dotted",label=r"$V_{Gas}$")
	plt.plot(data[:,0],data[:,4],marker='o',linestyle="dotted",label=r"$V_{Disk}$")
	plt.plot(data[:,0],data[:,5],marker='o',linestyle="dotted",label=r"$V_{Bulge}$")
	plt.xlabel("r (kpc)")
	plt.ylabel("V (km/s)")
	plt.title(names_list[i])
	plt.legend()
	#save and close
	figpath = os.path.join(dirOutPath+dirOut,figname)
	plt.savefig(figpath,dpi=300)
	plt.close()

## 2. Surface Brightness
The code plots the desired visualizations for each available galaxy and saves the results in a dedicated folder.<br>
This cell constructs surface brightness plots (SBdisk, SBbul) as reported in the `Rotmod_LTG` files.

In [16]:
#output settings
dirOutPath = "SPARCplots/"
dirOut = "Experimental_SurfaceBrightness"

#create output folder, if needed
os.makedirs(dirOutPath+dirOut, exist_ok=True)
#loop over dataset
for i in range(len(data_list)):
	data = data_list[i]
	figname = names_list[i]+"_surfBright"
	# figure
	plt.figure()
	plt.plot(data[:,0],data[:,-2],marker='o',linestyle="dotted", label=f"$\Sigma_{{Disk}}$")
	plt.plot(data[:,0],data[:,-1],marker='o',linestyle="dotted", label=f"$\Sigma_{{Bulge}}$")
	plt.xlabel("r (kpc)")
	plt.ylabel(f"$\Sigma$ (L/pc$^2$)")
	plt.title(names_list[i])
	plt.legend()
	# save and close
	figpath = os.path.join(dirOutPath+dirOut,figname)
	plt.savefig(figpath,dpi=300)
	plt.close()

# Retrieve Galaxy Index
---

In [17]:
#extract tabulated galaxy names
tableNames = [t["name"] for t in table]
#retrieve the galaxy row index in "table1.dat" file
tableNames.index("UGC02953")

109